In [2]:
# Our goal is to build a model for body mass of penguins
# We will partition the data, examine summary statistics, correlations and distributions
# then we will build an initial model and do some preliminary analysis
# finally, we'll evaluate our model using a test set

### Set up ----
# install any missing libraries

# !pip install palmerpenguins

# load libraries

# from palmerpenguins import load_penguins # This contains our data set,
# though we will manually upload it, below

import pandas as pd # data frame to store the data

from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score # for evaluation metrics

import matplotlib.pyplot as plt # for plotting
import seaborn as sns # for plotting pairs plot

import numpy as np # many mathematical operations and more, used for sqrt()

from mlxtend.feature_selection import SequentialFeatureSelector as SFS



In [ ]:
# specific Google Colab code to load csv data
from google.colab import files  # so we can upload a CSV to our notebook

uploaded = files.upload() # opens a file chooser to select our csv file


In [3]:
# Load Data
penguins = pd.read_csv('PalmerPenguins.csv')



In [4]:
# Step 1: Data pre-processing ----

# Drop rows with missing values for simplicity (we will learn to impute later)
penguins.dropna(inplace=True)

# Create categorical variable(s)
penguins['sex'] = pd.Categorical(penguins['sex'])

penguins = pd.get_dummies(penguins, columns=['sex'], drop_first=True)

# pd.get_dummies() creates a boolean variable, sex_male,
# we need it to be numeric
penguins['sex_male'] = penguins['sex_male'].astype(int)

In [5]:
### Step 2: Train/Test Split ----
train_data, test_data = train_test_split(
    penguins, # data set to split
    test_size=0.3, # proportion of observations to be in the test set
    random_state = 90210 # random seed for reproducibility
    )

In [ ]:
### Step 3: Data Exploration ----

# look at some of the data directly
train_data

# note, you'll be converting species to categorical during the exercise


In [ ]:

# calculate basic summary statistics
# note that this only describes numeric variables
train_data.describe()

In [ ]:
# Visualizations
sns.pairplot(train_data, hue='species', diag_kind='kde')
plt.show()


In [6]:
### Step 4: Feature Engineering ----

# pull out predictor variables
# (so we don't accidentally include something not helpful)
predictors = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'sex_male']

# separate X and Y variables
X_train = train_data[predictors].copy()
y_train = train_data['body_mass_g'].copy()

X_test = test_data[predictors].copy()
y_test = test_data['body_mass_g']

# Add quadratic terms to training set
X_train['bill_length_mm_sq'] = X_train['bill_length_mm'] ** 2
X_train['bill_depth_mm_sq'] = X_train['bill_depth_mm'] ** 2
X_train['flipper_length_mm_sq'] = X_train['flipper_length_mm'] ** 2

# Add quadratic terms to test set as well
X_test['bill_length_mm_sq'] = X_test['bill_length_mm'] ** 2
X_test['bill_depth_mm_sq'] = X_test['bill_depth_mm'] ** 2
X_test['flipper_length_mm_sq'] = X_test['flipper_length_mm'] ** 2


In [ ]:
# Let's look at our data again to see what we just did
X_train

In [ ]:
### Step 5: Feature & Model Selection ----

# Fit OLS Model with Scikit-learn
# Scikit-learn is focused on prediction only, limited summary information
model_skl = LinearRegression().fit(X_train, y_train)

# Print a table of coefficients
coef_table = pd.DataFrame({
    "Variable": ["Intercept"] + list(X_train.columns),
    "Coefficient": [model_skl.intercept_] + list(model_skl.coef_)
})

coef_table # prints table for viewing


In [ ]:
# Fit OLS Model with statsmodels
model_sm = sm.OLS(y_train, sm.add_constant(X_train)).fit()
print(model_sm.summary()) # wrapping this in print so it's easier to see

In [ ]:
# Automatic model selection
# Process: do with sklearn then take specification for inference with statsmodels

# Forward Selection
sfs_forward = SFS(LinearRegression(), # type of model
                  k_features=(1, 6), # will find between 1 and 6 features
                  forward=True, # search direction (backwards if False)
                  floating=False, # controls stepwise vs. one direction
                  scoring='neg_root_mean_squared_error', # find max RMSE
                  cv=5) # how many random subsets of data to test

sfs_forward = sfs_forward.fit(X_train, y_train)

best_subset = sfs_forward.subsets_[1]
for v in sfs_forward.subsets_.values():
    if v['avg_score'] > best_subset['avg_score']:
        best_subset = v


print(f'Best score: {- best_subset["avg_score"]:.2f}')
print(f'Best subset (indices): {best_subset["feature_idx"]}')
print(f'Best subset (names): {best_subset["feature_names"]}')

In [ ]:
# Backwards Selection
sfs_backward = SFS(LinearRegression(), # type of model
                  k_features=(1, 6), # will find between 1 and 6 features
                  forward=False, # search direction (backwards if False)
                  floating=False, # controls stepwise vs. one direction
                  scoring='neg_root_mean_squared_error', # find max RMSE
                  cv=5) # how many random subsets of data to test

sfs_backward = sfs_backward.fit(X_train, y_train)

best_subset = sfs_backward.subsets_[1]
for v in sfs_backward.subsets_.values():
    if v['avg_score'] > best_subset['avg_score']:
        best_subset = v


print(f'Best score: {- best_subset["avg_score"]:.2f}')
print(f'Best subset (indices): {best_subset["feature_idx"]}')
print(f'Best subset (names): {best_subset["feature_names"]}')


In [ ]:
# Stepwise Selection (Back and forth)
sfs_step = SFS(LinearRegression(), # type of model
                  k_features=(1, 6), # will find between 1 and 6 features
                  forward=True, # search direction (backwards if False)
                  floating=True, # controls stepwise vs. one direction
                  scoring='neg_root_mean_squared_error', # find max RMSE
                  cv=5) # how many random subsets of data to test

sfs_step = sfs_step.fit(X_train, y_train)

best_subset = sfs_step.subsets_[1]
for v in sfs_step.subsets_.values():
    if v['avg_score'] > best_subset['avg_score']:
        best_subset = v


print(f'Fest score: {- best_subset["avg_score"]:.2f}')
print(f'Best subset (indices): {best_subset["feature_idx"]}')
print(f'Best subset (names): {best_subset["feature_names"]}')

In [ ]:
# all three methods found the same subset
final_features = ['bill_depth_mm', 'flipper_length_mm', 'sex_male', 'bill_depth_mm_sq', 'flipper_length_mm_sq']

In [ ]:
# fit the final model with statsmodels for inference
final_model_sm = sm.OLS(y_train, sm.add_constant(X_train[final_features])).fit()

print(final_model_sm.summary())

In [ ]:
# sklearn final model
final_model_skl = LinearRegression().fit(X_train[final_features], y_train)

In [ ]:
### Step 6: Predictions and Final Evaluation ----

# scikit-learn
pred_skl = final_model_skl.predict(X_test[final_features])

r2_skl = r2_score(y_test, pred_skl)
rmse_skl = np.sqrt(mean_squared_error(y_test, pred_skl))

print("sklearn")
print(f"R-squared: {round(r2_skl, 2)}")
print(f"RMSE: {round(rmse_skl, 2)}")

# statsmodels
pred_sm = final_model_sm.predict(sm.add_constant(X_test[final_features]))

r2_sm = r2_score(y_test, pred_sm)
rmse_sm = np.sqrt(mean_squared_error(y_test, pred_sm))

print("\nstatsmodels")
print(f"R-squared: {round(r2_sm, 2)}")
print(f"RMSE: {round(rmse_sm, 2)}")

# Scatter Plot of Predictions
plt.scatter(y_test, pred_sm, alpha=0.7)
plt.xlabel("Actual Body Mass (g)")
plt.ylabel("Predicted Body Mass (g)")
plt.title("Predictions vs Actual")
plt.show()
